In [1]:
import pandas as pd 
import requests
import json
from tqdm import tqdm
import xmltodict
import re

In [2]:
access_token='1000.7c11968bb4ecdf127e388077772f483d.548ff27b12371930b5d5f8ac3217b965'

code="1000.862ebf2028c70db8919492963d1daa3f.eda4e1fbc5a49734a34b142109e050a3"

client_id="1000.CFDOHTVE8ZZDXJVRR3VHR7U9C3W1UT"

client_secret="30624b06180b20ab5252fc8e6145ad175762a367a0"

grant_type="authorization_code"

refresh_token="1000.178433f36c80bf5a4c44440b114afba7.93dfa314694f1e83749f44d81c045706"
scope=["Desk.tickets.ALL","Desk.tickets.UPDATE","Desk.settings.READ","Desk.basic.READ"]

In [3]:
URL = "https://accounts.zoho.com/oauth/v2/token"

params={"refresh_token":refresh_token,"client_id":client_id,"client_secret":client_secret,"scope":scope,"grant_type":"refresh_token"}

# sending get request and saving the response as response object

access_token_refresh = requests.post(url = URL,params=params)

access_token_refresh_dict = json.loads(access_token_refresh.text)

new_access_token=access_token_refresh_dict['access_token']

'id': '766285000041423003' #ID OF OPEN TICKET VIEW

In [4]:
list_of_froms=list(range(0,1000,100))

In [5]:
list_of_dicts=[]

org_id="786481962"
department_id_customer="766285000000006907"
view_id="766285000041423003"

for i in tqdm(list_of_froms):
    params = f"from={i}&limit=100&departmentIds={department_id_customer}&viewId={view_id}&sortBy=-customerResponseTime"

    headers = {
        "Authorization":f"Zoho-oauthtoken {new_access_token}",
        "orgId":org_id,
        "contentType": "application/json; charset=utf-8"
    }

    request = requests.get('https://desk.zoho.com/api/v1/tickets?'+params, headers=headers)

    data = json.loads(request.text)

    list_of_dicts = list_of_dicts + data["data"]

100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [00:31<00:00,  3.13s/it]


In [6]:
open_tickets=pd.DataFrame(list_of_dicts)

In [7]:
open_tickets

,id,ticketNumber,layoutId,email,phone,subject,status,statusType,createdTime,category,...,productId,assigneeId,teamId,channelCode,webUrl,isSpam,lastThread,customerResponseTime,isArchived,source
0,766285000042708228,235322,766285000000074011,accommodation.fft@seera.sa,None,3HLK4F,Open,Open,2023-08-24T14:59:11.000Z,None,...,None,None,None,None,https://support.eetglobal.net/support/eetsuppo...,False,"{'channel': 'EMAIL', 'isDraft': False, 'isForw...",2023-08-24T14:59:11.000Z,False,"{'appName': None, 'appPhotoURL': None, 'permal..."
1,766285000042618704,234829,766285000000074011,faiez.p@rezlive.com,None,REZ64E4F327 || 5B9L3X || 117975 / 117978 || ...,Open,Open,2023-08-24T12:29:33.000Z,Amend,...,None,766285000004999037,None,None,https://support.eetglobal.net/support/eetsuppo...,False,"{'channel': 'EMAIL', 'isDraft': False, 'isForw...",2023-08-24T14:56:13.000Z,False,"{'appName': None, 'appPhotoURL': None, 'permal..."
2,766285000042707193,235306,766285000000074011,vanessa.p@rezlive.com,None,XJ17PF //REZ64E2752F//SPECIAL REQUEST,Open,Open,2023-08-24T14:54:57.000Z,None,...,None,None,None,None,https://support.eetglobal.net/support/eetsuppo...,False,"{'channel': 'EMAIL', 'isDraft': False, 'isForw...",2023-08-24T14:54:57.000Z,False,"{'appName': None, 'appPhotoURL': None, 'permal..."
3,766285000042695532,235294,766285000000074011,support@withinearth.com,None,Hotel Request For Confirmation No. Mr.Sumiah A...,Open,Open,2023-08-24T14:52:01.000Z,Confirmation Number,...,None,None,None,None,https://support.eetglobal.net/support/eetsuppo...,False,"{'channel': 'EMAIL', 'isDraft': False, 'isForw...",2023-08-24T14:52:46.610Z,False,"{'appName': None, 'appPhotoURL': None, 'permal..."
4,766285000042697784,235289,766285000000074011,complaints@eetglobal.net,None,RE: Cancellation request #43QCSP,Open,Open,2023-08-24T14:50:14.000Z,None,...,None,None,None,None,https://support.eetglobal.net/support/eetsuppo...,False,"{'channel': 'EMAIL', 'isDraft': False, 'isForw...",2023-08-24T14:50:14.000Z,False,"{'appName': None, 'appPhotoURL': None, 'permal..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,766285000036318509,197118,766285000000074011,reservation@4allbooking.com,None,Re: [## 191085 ##] Hotel Confirmation Number R...,Open,Open,2023-08-08T07:33:56.000Z,None,...,None,None,None,None,https://support.eetglobal.net/support/eetsuppo...,False,"{'channel': 'EMAIL', 'isDraft': False, 'isForw...",2023-08-10T08:24:16.653Z,False,"{'appName': None, 'appPhotoURL': None, 'permal..."
996,766285000036323053,197121,766285000000074011,reservation@4allbooking.com,None,Re: [## 191097 ##] Hotel Confirmation Number R...,Open,Open,2023-08-08T07:34:28.000Z,None,...,None,None,None,None,https://support.eetglobal.net/support/eetsuppo...,False,"{'channel': 'EMAIL', 'isDraft': False, 'isForw...",2023-08-10T08:24:15.396Z,False,"{'appName': None, 'appPhotoURL': None, 'permal..."
997,766285000036315294,197083,766285000000074011,support@hojuzat.com,None,B62RVD / hcn / urgent,Open,Open,2023-08-08T07:22:49.000Z,Confirmation Number,...,None,None,None,None,https://support.eetglobal.net/support/eetsuppo...,False,"{'channel': 'EMAIL', 'isDraft': False, 'isForw...",2023-08-10T08:23:56.053Z,False,"{'appName': None, 'appPhotoURL': None, 'permal..."
998,766285000036341721,197298,766285000000074011,support@hojuzat.com,None,Re: 1SQP59 \\ HCN,Open,Open,2023-08-08T08:29:51.000Z,None,...,None,None,None,None,https://support.eetglobal.net/support/eetsuppo...,False,"{'channel': 'EMAIL', 'isDraft': False, 'isForw...",2023-08-10T08:23:54.897Z,False,"{'appName': None, 'appPhotoURL': None, 'permal..."


In [8]:
open_tickets['subject'].nunique()

785

In [9]:
open_tickets.drop_duplicates(['subject'],inplace=True, ignore_index=True)

In [10]:
def extract_codes(long_string):

    matches = re.findall(r'\b[A-Z0-9]{6}\b', long_string)
    if len(matches)!=0:
        return list(set(matches))
    else:
        return "_"
    
        


In [11]:
def checkin_date(lines):
    x=lines['BeginTravelDate']
    return x[:10]

In [12]:
def append_checkin_date_to_df(lines):
    f_list=[]
    data_dict={}
    try :
        z=[]
        for line in lines:
            z_1=checkin_date(line)
            z.append(z_1)
        #data_dict['booking_code']=code
        #data_dict['checkin_date']=z
        f_list.append(z) 
    except:   #one line in booking code
        #data_dict['booking_code']=code
        #data_dict['checkin_date']=checkin_date(lines)
        f_list.append(checkin_date(lines))
    return f_list


In [13]:
def is_code_check_in(x):
    try: #list of uncertin multi codes
        for code in x:
            url = "https://www.eetglobal.com/wsExportacion/wsBookings.asmx?op=getBookings"


            headers= {'content-type': 'text/xml','SOAPAction':"http://juniper.es/getBookings","User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:66.0) Gecko/20100101 Firefox/66.0",
                        "Accept-Encoding": "*","Connection": "keep-alive"}


            body = f"""<?xml version="1.0" encoding="utf-8"?>
                    <soap:Envelope xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xmlns:xsd="http://www.w3.org/2001/XMLSchema" xmlns:soap="http://schemas.xmlsoap.org/soap/envelope/">
                      <soap:Body>
                        <getBookings xmlns="http://juniper.es/">
                      <user>amr.ahmed</user>
                          <password>Eetglobal123456@</password>
                          <BookingCode>{code}</BookingCode>
                        </getBookings>
                      </soap:Body>
                    </soap:Envelope>"""
            response=requests.post(url=url , headers=headers , data=body)
            booking_info_dict=xmltodict.parse(response.text)
            Bookings=booking_info_dict['soap:Envelope']['soap:Body']['getBookingsResponse']['getBookingsResult']['wsResult'].get('Bookings')
            if Bookings==None:
                #print("not excist")
                pass 

            else:
                lines=Bookings['Booking']['Lines']['Line']
                return append_checkin_date_to_df(lines)
        
        
    except: #list of one uncertine code
        url = "https://www.eetglobal.com/wsExportacion/wsBookings.asmx?op=getBookings"


        headers= {'content-type': 'text/xml','SOAPAction':"http://juniper.es/getBookings","User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:66.0) Gecko/20100101 Firefox/66.0",
                    "Accept-Encoding": "*","Connection": "keep-alive"}


        body = f"""<?xml version="1.0" encoding="utf-8"?>
                <soap:Envelope xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xmlns:xsd="http://www.w3.org/2001/XMLSchema" xmlns:soap="http://schemas.xmlsoap.org/soap/envelope/">
                  <soap:Body>
                    <getBookings xmlns="http://juniper.es/">
                      <user>amr.ahmed</user>
                      <password>Eetglobal123456@</password>
                      <BookingCode>{x}</BookingCode>
                    </getBookings>
                  </soap:Body>
                </soap:Envelope>"""
        response=requests.post(url=url , headers=headers , data=body)
        booking_info_dict=xmltodict.parse(response.text)
        Bookings=booking_info_dict['soap:Envelope']['soap:Body']['getBookingsResponse']['getBookingsResult']['wsResult'].get('Bookings')
        if Bookings==None:
            #print("not excist")
            pass 

        else:
            lines=Bookings['Booking']['Lines']['Line']
            return append_checkin_date_to_df(lines)
             

In [14]:
open_tickets['booking']=open_tickets['subject'].apply(extract_codes)

In [15]:
open_tickets['booking']

0                      [3HLK4F]
1      [117978, 5B9L3X, 117975]
2                      [XJ17PF]
3                      [T7BL8R]
4                      [43QCSP]
                 ...           
780                    [KZGXCM]
781            [191085, 5XB3VT]
782            [8BL2B5, 191097]
783                    [B62RVD]
784                    [1SQP59]
Name: booking, Length: 785, dtype: object

In [16]:
index=open_tickets[open_tickets['booking']=="_"].index

In [17]:
open_tickets.drop(index=index, inplace=True)

In [18]:
open_tickets.reset_index(inplace=True,drop=True)

In [19]:
open_tickets

,id,ticketNumber,layoutId,email,phone,subject,status,statusType,createdTime,category,...,assigneeId,teamId,channelCode,webUrl,isSpam,lastThread,customerResponseTime,isArchived,source,booking
0,766285000042708228,235322,766285000000074011,accommodation.fft@seera.sa,None,3HLK4F,Open,Open,2023-08-24T14:59:11.000Z,None,...,None,None,None,https://support.eetglobal.net/support/eetsuppo...,False,"{'channel': 'EMAIL', 'isDraft': False, 'isForw...",2023-08-24T14:59:11.000Z,False,"{'appName': None, 'appPhotoURL': None, 'permal...",[3HLK4F]
1,766285000042618704,234829,766285000000074011,faiez.p@rezlive.com,None,REZ64E4F327 || 5B9L3X || 117975 / 117978 || ...,Open,Open,2023-08-24T12:29:33.000Z,Amend,...,766285000004999037,None,None,https://support.eetglobal.net/support/eetsuppo...,False,"{'channel': 'EMAIL', 'isDraft': False, 'isForw...",2023-08-24T14:56:13.000Z,False,"{'appName': None, 'appPhotoURL': None, 'permal...","[117978, 5B9L3X, 117975]"
2,766285000042707193,235306,766285000000074011,vanessa.p@rezlive.com,None,XJ17PF //REZ64E2752F//SPECIAL REQUEST,Open,Open,2023-08-24T14:54:57.000Z,None,...,None,None,None,https://support.eetglobal.net/support/eetsuppo...,False,"{'channel': 'EMAIL', 'isDraft': False, 'isForw...",2023-08-24T14:54:57.000Z,False,"{'appName': None, 'appPhotoURL': None, 'permal...",[XJ17PF]
3,766285000042695532,235294,766285000000074011,support@withinearth.com,None,Hotel Request For Confirmation No. Mr.Sumiah A...,Open,Open,2023-08-24T14:52:01.000Z,Confirmation Number,...,None,None,None,https://support.eetglobal.net/support/eetsuppo...,False,"{'channel': 'EMAIL', 'isDraft': False, 'isForw...",2023-08-24T14:52:46.610Z,False,"{'appName': None, 'appPhotoURL': None, 'permal...",[T7BL8R]
4,766285000042697784,235289,766285000000074011,complaints@eetglobal.net,None,RE: Cancellation request #43QCSP,Open,Open,2023-08-24T14:50:14.000Z,None,...,None,None,None,https://support.eetglobal.net/support/eetsuppo...,False,"{'channel': 'EMAIL', 'isDraft': False, 'isForw...",2023-08-24T14:50:14.000Z,False,"{'appName': None, 'appPhotoURL': None, 'permal...",[43QCSP]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
648,766285000036628842,199276,766285000000074011,ops1.eg@bookingexpress.co,None,RECONFIRM AND HCN -- KZGXCM -- 1 - 5 SEP -- BE...,Open,Open,2023-08-08T18:21:55.000Z,None,...,None,None,None,https://support.eetglobal.net/support/eetsuppo...,False,"{'channel': 'EMAIL', 'isDraft': False, 'isForw...",2023-08-10T08:24:19.809Z,False,"{'appName': None, 'appPhotoURL': None, 'permal...",[KZGXCM]
649,766285000036318509,197118,766285000000074011,reservation@4allbooking.com,None,Re: [## 191085 ##] Hotel Confirmation Number R...,Open,Open,2023-08-08T07:33:56.000Z,None,...,None,None,None,https://support.eetglobal.net/support/eetsuppo...,False,"{'channel': 'EMAIL', 'isDraft': False, 'isForw...",2023-08-10T08:24:16.653Z,False,"{'appName': None, 'appPhotoURL': None, 'permal...","[191085, 5XB3VT]"
650,766285000036323053,197121,766285000000074011,reservation@4allbooking.com,None,Re: [## 191097 ##] Hotel Confirmation Number R...,Open,Open,2023-08-08T07:34:28.000Z,None,...,None,None,None,https://support.eetglobal.net/support/eetsuppo...,False,"{'channel': 'EMAIL', 'isDraft': False, 'isForw...",2023-08-10T08:24:15.396Z,False,"{'appName': None, 'appPhotoURL': None, 'permal...","[8BL2B5, 191097]"
651,766285000036315294,197083,766285000000074011,support@hojuzat.com,None,B62RVD / hcn / urgent,Open,Open,2023-08-08T07:22:49.000Z,Confirmation Number,...,None,None,None,https://support.eetglobal.net/support/eetsuppo...,False,"{'channel': 'EMAIL', 'isDraft': False, 'isForw...",2023-08-10T08:23:56.053Z,False,"{'appName': None, 'appPhotoURL': None, 'permal...",[B62RVD]


In [20]:
df=open_tickets[['subject', 'booking']]

In [21]:
df['check in']=df['booking'].apply(is_code_check_in)

C:\Users\int3\AppData\Local\Temp\ipykernel_13544\2332098705.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['check in']=df['booking'].apply(is_code_check_in)


In [22]:
df

,subject,booking,check in
0,3HLK4F,[3HLK4F],[2023-08-24]
1,REZ64E4F327 || 5B9L3X || 117975 / 117978 || ...,"[117978, 5B9L3X, 117975]",[2023-08-23]
2,XJ17PF //REZ64E2752F//SPECIAL REQUEST,[XJ17PF],[2023-08-24]
3,Hotel Request For Confirmation No. Mr.Sumiah A...,[T7BL8R],[2023-08-28]
4,RE: Cancellation request #43QCSP,[43QCSP],[2023-08-23]
...,...,...,...
648,RECONFIRM AND HCN -- KZGXCM -- 1 - 5 SEP -- BE...,[KZGXCM],[2023-09-01]
649,Re: [## 191085 ##] Hotel Confirmation Number R...,"[191085, 5XB3VT]",[2023-08-19]
650,Re: [## 191097 ##] Hotel Confirmation Number R...,"[8BL2B5, 191097]",[2023-08-15]
651,B62RVD / hcn / urgent,[B62RVD],[2023-08-08]


In [23]:
df.to_excel("ticket subject result", index=False)

ValueError: No engine for filetype: ''